# Sesión 3 Databricks - Análisis Exploratorio e Introducción a Unity Catalog


## Creacion entorno

Vamos, primero de todo, a crear el entorno en el cual vamos a trabajar

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS sesion3;
CREATE SCHEMA IF NOT EXISTS sesion3.data COMMENT 'This is customer catalog';
CREATE VOLUME IF NOT EXISTS sesion3.data.landing;

DESCRIBE SCHEMA EXTENDED sesion3.data;

In [0]:
%sh
curl -L https://raw.githubusercontent.com/dvddepennde/crops_training_school/refs/heads/main/nutrients_csvfile.csv -o /Volumes/sesion3/data/landing/nutrients_csvfile.csv

In [0]:
import requests
# Url de output
output_path = f"/Volumes/sesion3/data/landing/data_demo.csv"

# URL del archivo CSV
csv_url = "https://raw.githubusercontent.com/vbelz/Outliers_detection/refs/heads/main/data/data_demo.csv"

# Realizar la solicitud GET para obtener el archivo
response = requests.get(csv_url)

# Verificar que la solicitud fue exitosa (código 200)
if response.status_code == 200:
    # Guardar el archivo en la ruta de destino
    with open(output_path, 'wb') as f:
        f.write(response.content)
    print(f"Archivo descargado exitosamente en: {output_path}")
else:
    print(f"Error al descargar el archivo. Código de estado: {response.status_code}")

In [0]:
path_data_demo = "/Volumes/sesion3/data/landing/data_demo.csv"
df = spark.read.csv(
    path=path_data_demo,
    header=True,     
    inferSchema=True
)
df.write.mode("overwrite").saveAsTable("sesion3.data.casas")
display(df)

## Análisis exploratorio con Spark

### Información General del Dataset

- **Dimensiones**: Indican la cantidad de observaciones y variables.
- **Tipos de datos**: Es importante verificar que los tipos de datos sean los esperados (por ejemplo, numéricos para variables cuantitativas).
- **Primeras filas**: Visualizar las primeras filas ayuda a entender la estructura y contenido de los datos.


In [0]:
# Número de filas y columnas
print(f"Dimensiones del DataFrame: {df.count()} filas, {len(df.columns)} columnas")

# Tipos de datos
df.printSchema()

# Mostrar las primeras filas
df.show(5)


In [0]:
rows = df.count()
columns = len(df.columns)
print(f"Número de filas: {rows}, columnas: {columns}")


Como podemos observar, todas las columnas son numéricas y no hay que realizar ningún tipo de conversión. Es posible que en algún momento no lo sean y toque realizar transformaciones oportunas para poder realizar correctamente el análisis exploratorio de los datos.

In [0]:
from pyspark.sql.functions import isnan, when, count, col

display(
    df.select([
    count(
        when(
            isnan(c), c
        )
    ).alias(c) for c in df.columns])
)

El método describe() proporciona estadísticas básicas como el conteo, media, desviación estándar, mínimo y máximo para cada columna numérica del DataFrame.

In [0]:
df.describe().show()

Este comando generará una tabla con las siguientes estadísticas para cada columna numérica:​
- count: Número de valores no nulos.
- mean: Media aritmética.
- stddev: Desviación estándar.
- min: Valor mínimo.
- max: Valor máximo

El método summary() ofrece un conjunto más amplio de estadísticas, incluyendo percentiles y la mediana

In [0]:
df.summary().show()

### Matriz de correlación 

rimero, necesitas convertir las columnas numéricas en un único vector utilizando VectorAssembler, ya que el método de correlación de PySpark requiere una columna de tipo Vector y posteriormente utiliza la clase Correlation de pyspark.ml.stat para calcular la matriz de correlación de Pearson

In [0]:

import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
df_pd = df.toPandas()
pd.set_option('display.max_columns', None)

plt.figure(figsize=(10, 8))
mask_heatmap = np.triu(np.ones_like(df_pd.corr(), dtype=bool))
sns.heatmap(data=df_pd.corr(), annot=True, linewidth=3, cmap='Blues', mask=mask_heatmap)
plt.show()

### Outliers: Sobre la columna "time_on_market"

Un outlier (o valor atípico) es un punto de datos que se desvía significativamente del resto de las observaciones en un conjunto de datos. Estos valores pueden surgir por errores de medición, variaciones naturales o eventos poco comunes. Detectar y tratar los outliers es crucial, ya que pueden afectar negativamente la calidad de los modelos predictivos, sesgar estadísticas descriptivas y ocultar patrones importantes.​

#### Rango Intercuartilico IQR
El IQR mide la dispersión entre el primer (Q1) y el tercer cuartil (Q3) de los datos. Este método es robusto frente a distribuciones sesgadas y no requiere suposiciones sobre la forma de los datos

In [0]:
from pyspark.sql import functions as F

# Calcular Q1, Q3 e IQR
quantiles = df.approxQuantile("time_on_market", [0.25, 0.75], 0.05)
q1, q3 = quantiles
iqr = q3 - q1

# Definir límites
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

outliers = df.filter((F.col("time_on_market") < lower_bound) | (F.col("time_on_market") > upper_bound))

# Mostrar los outliers
display(outliers)


#### Z-Score 
La puntuación Z indica cuántas desviaciones estándar se aleja un punto de la media. Valores con una Z-score mayor a 3 o menor a -3 suelen considerarse outliers.

In [0]:
from pyspark.sql import functions as F

# Calcular la media y la desviación estándar
mean_stddev = df.select(
    F.mean("time_on_market").alias("mean"),
    F.stddev("time_on_market").alias("stddev")
).first()

mean_value = mean_stddev["mean"]
stddev_value = mean_stddev["stddev"]

# Calcular el Z-score
df_with_zscore = df.withColumn(
    "zscore_time_on_market",
    (F.col("time_on_market") - mean_value) / stddev_value
)

# Filtrar los outliers
outliers = df_with_zscore.filter(
    (F.abs(F.col("zscore_time_on_market")) > 3)
)

# Mostrar los outliers
display(outliers)

#### Detección visual

En este caso, vamos a verboxplot y scatterplot:

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convertir a Pandas
df_pd = df.select(['time_on_market']).toPandas()

# Boxplot
plt.figure(figsize=(12, 4))
sns.boxplot(data=df_pd)
plt.title("Boxplot de Variable time_on_market")
plt.show()




In [0]:
# Convert to Pandas (only once)
df_pd = df.select(['rooms']).toPandas()

# Create the pairplot
plt.figure(figsize=(12, 4))
sns.pairplot(df_pd, diag_kind='kde')
plt.show()


### Ejercicios
1. Realiza analisis exploratorio sobre algun dataset de tu elección. Ten en cuenta que tendrás que, descargarlo y cargarlo como DataFrame. En caso de no saber de ninguno, podréis usar el siguiente: https://raw.githubusercontent.com/Saba-Gul/Exploratory-Data-Analysis-and-Statistical-Analysis-Notebooks/refs/heads/main/Datasets/heart_failure_clinical_records%20-%20heart_failure_clinical_records.csv

2. En base al dataset escogido en el anterior paso, realiza la detección de outliers con los 3 métodos enseñados

## Analisis exploratorio SQL

In [0]:
%sql
-- 1.1 Ver filas de ejemplo
SELECT * 
FROM sesion3.data.casas
LIMIT 10;


In [0]:
%sql
-- 1.2 Número total de registros
SELECT COUNT(*) AS total_registros
FROM sesion3.data.casas;

In [0]:
%sql
-- 1.3 Número de columnas y esquema
DESCRIBE sesion3.data.casas;

In [0]:
%sql
-- 2.1 Estadísticas clásicas (min, max, avg, stddev)
SELECT
  MIN(value)    AS min_value,
  MAX(value)    AS max_value,
  AVG(value)    AS avg_value,
  STDDEV(value) AS stddev_value,
  MIN(time_on_market)    AS min_tom,
  MAX(time_on_market)    AS max_tom,
  AVG(time_on_market)    AS avg_tom,
  STDDEV(time_on_market) AS stddev_tom
FROM sesion3.data.casas;

In [0]:
%sql
-- 2.2 Percentiles
SELECT
  percentile_approx(value, 0.25) AS q1_value,
  percentile_approx(value, 0.50) AS median_value,
  percentile_approx(value, 0.75) AS q3_value
FROM sesion3.data.casas;

In [0]:
%sql
-- Conteo de nulos
SELECT
  SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END)            AS null_value,
  SUM(CASE WHEN time_on_market IS NULL THEN 1 ELSE 0 END)   AS null_time_on_market,
  SUM(CASE WHEN interior_quality IS NULL THEN 1 ELSE 0 END) AS null_quality
FROM sesion3.data.casas;


In [0]:
%sql 
-- 4.1 Calcular IQR y límites
WITH stats AS (
  SELECT
    percentile_approx(value, 0.25) AS q1,
    percentile_approx(value, 0.75) AS q3
  FROM sesion3.data.casas
),
bounds AS (
  SELECT
    q1,
    q3,
    (q3 - q1) AS iqr,
    (q1 - 1.5*(q3-q1)) AS lower_bound,
    (q3 + 1.5*(q3-q1)) AS upper_bound
  FROM stats
)
-- 4.2 Identificar outliers
SELECT c.*
FROM sesion3.data.casas c
CROSS JOIN bounds b
WHERE c.value < b.lower_bound
   OR c.value > b.upper_bound;


In [0]:
%sql
-- 5.1 Calcular media y desviación estándar
WITH mu_sigma AS (
  SELECT
    AVG(time_on_market)    AS mu,
    STDDEV(time_on_market) AS sigma
  FROM sesion3.data.casas
)
-- 5.2 Calcular Z-score y filtrar
SELECT
  c.*,
  (c.time_on_market - m.mu)/m.sigma AS zscore_tom
FROM sesion3.data.casas c
CROSS JOIN mu_sigma m
WHERE ABS((c.time_on_market - m.mu)/m.sigma) > 3;


In [0]:
%sql 
-- Correlaciones
SELECT 
  corr(useful_area, value)           AS corr_area_value,
  corr(time_on_market, value)        AS corr_tom_value,
  corr(interior_quality, value)      AS corr_quality_value
FROM sesion3.data.casas;


### Ejercicio
1. Realiza alguna visualización de tipo Bar, Pie e Histogram. Realizalo sobre el conjunto elegido por ti en el conjunto de ejercicios anterior. Coge de la parte inicial del notebook como guardar el dataframe como tabla para poder realizar queries sobre él de forma cómoda.